In [2]:
import json, time, random

from typing import Dict, Any
from base64 import b64decode

from algosdk import account, transaction, mnemonic
from algosdk.v2client import algod

### Connect Algorand Testnet

In [3]:
# setup algod client
algod_address = "https://testnet-api.algonode.cloud"
algod_token = ""

algod_client = algod.AlgodClient(algod_token, algod_address)

### Create accounts 

In [4]:
# liquidity provider, trader, and liquidity pool address
liquidity_provider_private_key, liquidity_provider_address         = account.generate_account()
trader_private_key, trader_address                                 = account.generate_account()
pool_private_key, pool_address                                     = account.generate_account()


print(f"Liquidity provider Address: {liquidity_provider_address}")
print(f"Trader Address: {trader_address}")
print(f" Liquidity Pool Address: {pool_address}")



Liquidity provider Address: WV2PSLI62SMRPMBKM4P7VNYR6RGFUEDAKRZXP4Q55W4JGEFHP64HWELM7U
Trader Address: U54N5D62J3UNSUGW5PK43BPIHZSL55GWPLRSJ6Y6W2YB3M36IXQEUR5QKY
 Liquidity Pool Address: 4ZZXDLONSBHB4JMUESARFMXGRZNGYQA3XDSIP5JNEI2H3YVPSUY5CTUCJY


### Fund the accounts with enough Algos each. 10 Algos can do. 

In [ ]:
user_input = input(
    "Now go to Algorand testnet dispenser (https://dispenser.testnet.aws.algodev.network/) and fund stokvel accounts, then return and press enter. "
)

### Let's create some UCTZAR stablecoin 

In [5]:

# a function is better to use ;) 
def create_asset(private_key, asset_name, unit_name, total_units):

    address = account.address_from_private_key(private_key)
    params = algod_client.suggested_params()
    
    txn = transaction.AssetConfigTxn(
        sender=address,
        sp=params,
        total=total_units,                # Total number of units of this asset
        decimals=0,                       # No decimal places
        default_frozen=False,             # Asset is not frozen by default
        unit_name=unit_name,              # Short name for the asset (e.g., "ALGO")
        asset_name=asset_name,            # Full name for the asset
        manager=address,                  # Account that can manage the asset 
        reserve=address,                  # Account that holds the reserve 
        freeze=address,                   # Account that can freeze the asset
        clawback=address                  # Account that can clawback the asset 
    )

    # Sign and send the transaction
    signed_txn = txn.sign(private_key)
    tx_id      = algod_client.send_transaction(signed_txn)
    print(f"Transaction ID for asset creation ({asset_name}): {tx_id}")

    # Wait for the transaction to be confirmed
    try:
        confirmed_txn = transaction.wait_for_confirmation(algod_client, tx_id, 4)
        asset_id = confirmed_txn["asset-index"]
        print(f"Created asset {asset_name} with Asset ID: {asset_id}")
        return asset_id
    except Exception as e:
        print(e)
        return None
    


# UCTZAR Asset 
uctzar_asset_id = create_asset(liquidity_provider_private_key,
                                asset_name= "UCTZAR Token", 
                                unit_name= "UCTZAR", 
                                total_units= 10000000)





Transaction ID for asset creation (UCTZAR Token): AWVVLVVNOYL6CDWAG64Q35UZRM46BV2BVUFDBF2PRPAD4DZNIFQA
Created asset UCTZAR Token with Asset ID: 728821715


### Opt-in to the UCTZAR Asset 

For the trader and the liquidity pool to receive and trade UCTZAR, they need to “opt-in” to the asset. Opting in to an asset creates a balance entry in the account for this asset.

In [7]:
def opt_in_asset(asset_id, address, private_key):
    params = algod_client.suggested_params()
    txn = transaction.AssetTransferTxn(
        sender=address,
        sp=params,
        receiver=address,
        amt=0,  # Opt-in requires a 0 amount transfer to self
        index=asset_id
    )
    
    # Sign and send the transaction
    signed_txn = txn.sign(private_key)
    tx_id = algod_client.send_transaction(signed_txn)
    print(f"Opt-in Transaction ID for asset {asset_id}: {tx_id}")

    # Wait for confirmation
    try:
        confirmed_txn = transaction.wait_for_confirmation(algod_client, tx_id, 4)
        print("Opt-in confirmed for Asset ID:", asset_id)
    except Exception as e:
        print(e)


# Trader and liquidity account opt in to UCTZAR
opt_in_asset(asset_id = uctzar_asset_id, private_key = trader_private_key, address = trader_address)
opt_in_asset(uctzar_asset_id, private_key =  pool_private_key, address = pool_address)

Opt-in Transaction ID for asset 728821715: DPA6C6VS3JTWZKRJAHH5KFNAGUIBDQFX6XF4IS5X3BXKGMQHGLQA
Opt-in confirmed for Asset ID: 728821715
Opt-in Transaction ID for asset 728821715: WRHUWBZKAAKBCLXW63OI32Y7PSTEGTMPJ5KOERN5LWUOJA4RVNMQ
Opt-in confirmed for Asset ID: 728821715


### Distribute UCTZAR to the trader account to begin simulation 

In [ ]:
def transfer_asset(asset_id, sender_private_key, receiver_address, amount):

    sender_address = account.address_from_private_key(sender_private_key)
    params         = algod_client.suggested_params()

    txn = transaction.AssetTransferTxn(
        sender  =sender_address,
        sp      =params,
        receiver=receiver_address,
        amt     =amount,
        index   =asset_id
    )

    # Sign and send the transaction
    signed_txn = txn.sign(sender_private_key)
    tx_id      = algod_client.send_transaction(signed_txn)
    print(f"Transfer Transaction ID: {tx_id}")

    # Wait for confirmation
    try:
        confirmed_txn = transaction.wait_for_confirmation(algod_client, tx_id, 4)
        print(f"Transferred {amount} of asset ID {asset_id} to {receiver_address}")
    except Exception as e:
        print(e)

# Transfer UCTZAR from Liquidity Provider to Trader
transfer_asset(asset_id          = uctzar_asset_id,
              sender_private_key = liquidity_provider_private_key,
               receiver_address  = trader_address,
                amount           =  100000)  # 10000 UCTZAR





Transfer Transaction ID: FHAK6NUS34WPLMSPRE2FOMEHJMPO24B6NWI4MKYHCNNQQXYEVRNQ
Transferred 100000 of asset ID 728821715 to TKJZRMHGMYAZR5VYFAE7LG4D3P6J5V3AOY2IOVWY7ENAZIDEIVJ3ZODGGE


### Depositing Liquidity and Facilitating Trades 

In a decentralized liquidity pool, we have two primary functions:
1. Deposit Liquidity: The liquidity provider deposits a pair of tokens (ALGO and UCTZAR) into the pool to provide liquidity. They subsequently recieve their staking. I assumed that the staking will be 10% of the fees for each trade. 
2. Trade Execution: A trader can swap one token for the other (e.g., trading ALGO for UCTZAR or vice versa).


In [ ]:
# Deposit liquidity 
def deposit_liquidity(algo_amount, uctzar_amount, lp_shareholder, provider_private_key):
    """
    Deposit ALGO and UCTZAR into the liquidity pool.
    
    
    :param algo_amount: Amount of ALGO to deposit
    :param uctzar_amount: Amount of UCTZAR to deposit
    :param lp_shareholder: Address of liquidity provider
    :param provider_private_key: Private key of the liquidity provider
    :return: LP shares issued
    """
    # Ensure deposits are in the correct ratio: 1 UCTZAR = 0.5 ALGO
    if uctzar_amount != 2 * algo_amount:
        print("Deposit amounts do not match the required 1 UCTZAR = 0.5 ALGO ratio.")
        
    
    # Retrieve suggested transaction parameters
    params = algod_client.suggested_params()
    
    # Liquidity provider deposits algos
    algo_txn = transaction.PaymentTxn(
        sender   =lp_shareholder,
        receiver = pool_address,  # address of the liquidity pool 
        amt      = algo_amount,
        sp       =params
    )
    
    # UCTZAR deposit (asset transfer)
    uctzar_txn = transaction.AssetTransferTxn(
        sender   = lp_shareholder,
        receiver = pool_address,
        amt      = uctzar_amount,
        index    = uctzar_asset_id,  # Asset ID of UCTZAR
        sp       = params,
    )
    
    # sign and group the transactions 
    transaction.assign_group_id([algo_txn,uctzar_txn])
    signed_algo_txn   = algo_txn.sign(provider_private_key)
    signed_uctzar_txn = uctzar_txn.sign(provider_private_key)

    

    
    # Send transactions
    tx_id = algod_client.send_transactions([signed_algo_txn, signed_uctzar_txn])
    print(f"Liquidity deposit transaction ID: {tx_id}")
    
    # Wait for confirmation
    confirmed_txn = transaction.wait_for_confirmation(algod_client, tx_id, 4)
    
    # Issue LP Shares
    lp_shares = int((algo_amount)* 0.001)
    print(f"You are entitled to 10% of every trading fee! You can claim them once you wish to surrend these shares.")


# Executing the trade function 
def execute_trade(trade, trader_address, trader_private_key,user_said ):
    """
    Perform a trade in the liquidity pool.

    
    :param trade: Amount of ALGO/UCTZAR the trader provides
    :param trader_address: Address of the trader
    :param trader_private_key: Private key of the trader
    :param user_said: What do you want to trade for ? UCTZAR or ALGO
    :return: UCTZAR output for the given ALGO input
    """

    # Calculate trading fee (e.g., 0.5% fee)
    fee             = int(trade * 0.005)
    lp_fee        = fee*0.1
    trade_after_fee = trade - fee





    # Suggested transaction parameters
    params = algod_client.suggested_params()

    if user_said == "UCTZAR":

        uctzar_out = trade*2

        # ALGO transfer from trader to pool
        algo_txn = transaction.PaymentTxn(
        sender  =trader_address,
        receiver= pool_address,
        amt= trade_after_fee,
        sp=params
        )
    
        # UCTZAR transfer from pool to trader
        uctzar_txn = transaction.AssetTransferTxn(
            sender  = pool_address,
            sp      =params,
            receiver=trader_address,
            amt     =uctzar_out,
            index   =uctzar_asset_id
        )

        # sign and group the transactions 
        transaction.assign_group_id([algo_txn, uctzar_txn])
        signed_algo_txn   = algo_txn.sign(trader_private_key)
        signed_uctzar_txn = uctzar_txn.sign(pool_private_key)

    
        # Send transactions
        tx_id = algod_client.send_transactions([signed_algo_txn, signed_uctzar_txn])
        print(f"Trade transaction ID: {tx_id}")

        # Wait for confirmation
        confirmed_txn = transaction.wait_for_confirmation(algod_client, tx_id, 4)
        print(f"Trade executed: {trade} for {uctzar_out}")

        return {"liquidity provider share received": lp_fee}
        
    else:

        algo_out = int(trade/2)

        # Algos transfer 
        algo_txn = transaction.PaymentTxn(
        sender  = pool_address,
        receiver= trader_address,
        amt    = algo_out,
        sp     =params
        )
    
        # UCTZAR transfer
        uctzar_txn = transaction.AssetTransferTxn(
            sender  = trader_address,
            sp      =params,
            receiver=pool_address ,
            amt     = trade_after_fee,
            index   =uctzar_asset_id
        )

        # sign and group the transactions 
        transaction.assign_group_id([algo_txn, uctzar_txn])
        signed_algo_txn   = algo_txn.sign(pool_private_key)
        signed_uctzar_txn = uctzar_txn.sign(trader_private_key)

        

        # Send transactions
        tx_id = algod_client.send_transactions([signed_algo_txn, signed_uctzar_txn])
        print(f"Trade transaction ID: {tx_id}")

        # Wait for confirmation
        confirmed_txn = transaction.wait_for_confirmation(algod_client, tx_id, 4)
        print(f"Trade executed: {trade} for {algo_out}")

        return {"liquidity provider share received": lp_fee}
    
    



#### Here's one iteration of the liquidity pooling on a DEX

In [37]:
deposit_liquidity(algo_amount   = 100000, 
                  uctzar_amount = 200000,
                  lp_shareholder= liquidity_provider_address, 
                  provider_private_key= liquidity_provider_private_key
                  )

execute_trade(trade = 10000, 
              trader_address= trader_address,
              trader_private_key= trader_private_key,
              user_said= "ALGO")

Liquidity deposit transaction ID: 5MD52NXDHKMOLMVDBUZ5V2L6LYGI6LWTDLPQLH6VETBI5BG2SXFQ
You are entitled to 10% of every trading fee! You can claim them once you wish to surrend these shares.
Trade transaction ID: EGVDCAQP4C7ZSGBETVAM3T65CQPLHF74RB5YOKTCVXSZ6QEZG3VQ
Trade executed: 10000 for 5000


{'liquidity provider share received': 5.0}

##### just need to figure out how to surrender the liquidity share token recieved. 